In [2]:
import pandas as pd
import numpy as np
import googlemaps
import gmaps
import gmaps.datasets
import geopandas as gpd
#import matplotlib as plt
import matplotlib.pyplot as plt
import os

from config import g_key
gmaps.configure(api_key = g_key)

ImportError: DLL load failed: The specified module could not be found.

In [ ]:
data = pd.read_csv('source_files/Fire_Data_for_mapping.csv', encoding = "ISO-8859-1", low_memory=False, index_col = 'OBJECTID')
data = data.fillna('')                   # fill empty entries with ''
print(list(data))                        # print Variable Name
data.head()                              # show some data

In [ ]:
center_lat = '35.787743'
center_long = '-78.644257'
print('center = ',center_lat,center_long)

In [ ]:
fire_loc = pd.DataFrame(data)
fire_loc = fire_loc[fire_loc.Latitude != 'missing']
fire_loc['Latitude'] = pd.to_numeric(fire_loc['Latitude'])
fire_loc['Longitude'] = pd.to_numeric(fire_loc['Longitude'])

fire_loc.head()

In [ ]:
len(fire_loc)

In [ ]:
locations = fire_loc[['Latitude', 'Longitude']]
fig = gmaps.figure(map_type='TERRAIN')
heatmap_layer = gmaps.heatmap_layer(locations)
heatmap_layer.max_intensity = 50
heatmap_layer.point_radius = 5
fig.add_layer(gmaps.heatmap_layer(locations))
fig

In [ ]:
fp = 'Shp_files/Zip_Codes.shp'

map_df = gpd.read_file(fp)
map_df.head()

In [ ]:
map_df.plot(figsize=(10, 8));


In [ ]:
locs = pd.read_csv('Source_files/Fire_Data_for_mapping.csv', header=0, low_memory=False)

locs.head()

In [ ]:
df = locs.loc[locs['Incident']=='Fire']
df.head()

In [ ]:
len(df)

In [ ]:
#merged = map_df.set_index('NAME').join(df.set_index('Incident'))
#merged_df = pd.merge(df, map_df, on='OBJECTID', how='left')
merged_df = pd.concat([df, map_df], ignore_index=True, sort=False)
merged_df.head()

In [ ]:
len(merged_df)

In [ ]:
# set a variable that will call column to visualise on the map

variable = 'OBJECTID'

# set the range for the choropleth

vmin, vmax = 50, 2000

# create figure and axes for Matplotlib

fig, ax = plt.subplots(1, figsize=(15, 15))

# create map

merged.plot(column=variable, cmap='Reds', linewidth=0.8, ax=ax, edgecolor='0.8');

ax.axis('off')

ax.set_title('Fire Incident Rate in Wake County', fontdict={'fontsize': '25', 'fontweight' : '3'})

# Create colorbar as a legend

sm = plt.cm.ScalarMappable(cmap='Reds', norm=plt.Normalize(vmin=vmin, vmax=vmax))

# empty array for the data range

sm._A = []

# add the colorbar to the figure

cbar = fig.colorbar(sm)

ax.annotate('2008-2018',
            xy=(0.001, .225), xycoords='figure fraction',
            horizontalalignment='left', verticalalignment='top',
            fontsize=35)

In [ ]:
fig.savefig("Fire Incident Rate in Wake County 2008-2018.png", dpi=300)

In [ ]:
output_path = 'Maps'

# counter for the for loop
i = 0

# list of years
list_of_years = ['2008','2009','2010','2011','2012','2013','2014','2015','2016', '2017', '2018']

vmin, vmax = 5, 100


for year in list_of_years:
    
    plot_year=merged_df.loc[merged_df['Year']==int(year)]
    geo_plot_year = gpd.GeoDataFrame(plot_year)
    fig = merged.plot(column=variable, cmap='Reds', figsize=(12,12),
                         linewidth=0.8, edgecolor='0.8', vmin=vmin, vmax=vmax,
                         legend=True, norm=plt.Normalize(vmin=vmin, vmax=vmax))
    
    # removes axis
    fig.axis('off')
    
    # add a title
    fig.set_title('Fire Incidents in Wake County', \
              fontdict={'fontsize': '25',
                         'fontweight' : '3'})
    # add an annotation
    fig.annotate(int(year),
            xy=(0.1, .225), xycoords='figure fraction',
            horizontalalignment='left', verticalalignment='top',
            fontsize=35)
    
    filepath = os.path.join(output_path, year+'_fires.jpg')
    chart = fig.get_figure()
    chart.savefig(filepath, dpi=300);
    